In [2]:
from salad.serialize import read

In [3]:
fakes = read("/epyc/projects/salad/search/new2/DEEP/20190403/A0c/detector_1/fakes.pkl")
catalog = read("/epyc/projects/salad/search/new2/DEEP/20190403/A0c/detector_1/snr_3.0/regular/catalog.pkl")
catalog_no_masks = read("/epyc/projects/salad/search/new2/DEEP/20190403/A0c/detector_1/snr_3.0/regular/catalog_no_masks.pkl")


In [4]:
def recover(catalog, fakes, match_threshold_points=1/3600):
    import astropy.table
    from astropy.coordinates import SkyCoord
    import astropy.units as u
    
    matches = {orbit : {"points": {}, "line": {}, "catalog": 0, "info": {}} for orbit in set(fakes['ORBITID'])}
    c_fake = SkyCoord(fakes['RA']*u.deg, fakes['DEC']*u.deg)
    t1 = astropy.table.Table(data=[c_fake.ra, c_fake.dec, fakes['ORBITID'], fakes['EXPNUM']], names=['ra', 'dec', 'orbit', 'expnum'])
    
    x = catalog.X(columns=['ra', 'dec', 'time', 'exposures'])
    c_catalog = SkyCoord(x[:, 0]*u.deg, x[:, 1]*u.deg)
    t_catalog = astropy.table.Table(
        data=[c_catalog.ra, c_catalog.dec, x[:, 3]], 
        names=['ra', 'dec', 'expnum']
    )

    for g in astropy.table.join(t1, t_catalog, keys=['expnum']).group_by(["orbit"]).groups:
        sep = SkyCoord(g['ra_1'], g['dec_1']).separation(SkyCoord(g['ra_2'], g['dec_2']))
        n_matches = (sep.value < match_threshold_points).sum()
        o = int(g[0]['orbit'])
        matches[o]['catalog'] = {
            "fake": n_matches, # number of matches to fakes
            "total": len(fakes[fakes['ORBITID'] == o]), # number of injected fakes
        }

    return matches

In [5]:
c = recover(catalog, fakes)
cn = recover(catalog_no_masks, fakes)

In [14]:
_c = 0
_cn = 0
for k in c:
    print(c[k]['catalog'], cn[k]['catalog'])
    _c += c[k]['catalog']['fake']
    _cn += cn[k]['catalog']['fake']

{'fake': 1, 'total': 104} {'fake': 1, 'total': 104}
{'fake': 1, 'total': 104} {'fake': 2, 'total': 104}
{'fake': 0, 'total': 21} {'fake': 0, 'total': 21}
{'fake': 0, 'total': 104} {'fake': 5, 'total': 104}
{'fake': 85, 'total': 104} {'fake': 104, 'total': 104}
{'fake': 3, 'total': 104} {'fake': 3, 'total': 104}
{'fake': 0, 'total': 104} {'fake': 0, 'total': 104}
{'fake': 72, 'total': 104} {'fake': 92, 'total': 104}
{'fake': 11, 'total': 104} {'fake': 11, 'total': 104}
{'fake': 0, 'total': 104} {'fake': 1, 'total': 104}
{'fake': 61, 'total': 104} {'fake': 64, 'total': 104}
{'fake': 1, 'total': 104} {'fake': 1, 'total': 104}
{'fake': 100, 'total': 104} {'fake': 101, 'total': 104}
{'fake': 19, 'total': 104} {'fake': 19, 'total': 104}
{'fake': 20, 'total': 104} {'fake': 20, 'total': 104}
{'fake': 0, 'total': 104} {'fake': 2, 'total': 104}


In [7]:
catalog.mask_summary['masked'].sum() / catalog.mask_summary['total'].sum()

0.1401200810304055

In [15]:
(_cn - _c) / _cn 
# the fraction of lost detections is about the same as the fraction of masked pixels
# so I should compute completeness 

0.12206572769953052